In [20]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 18: Operation Order ---</h2><p>As you look out the window and notice a heavily-forested continent slowly appear over the horizon, you are interrupted by the child sitting next to you. They're curious if you could help them with their <span title="Or &quot;maths&quot;, if you have more than one.">math</span> homework.</p>
<p>Unfortunately, it seems like this "math" <a href="https://www.youtube.com/watch?v=3QtRK7Y2pPU&amp;t=15" target="_blank">follows different rules</a> than you remember.</p>
<p>The homework (your puzzle input) consists of a series of expressions that consist of addition (<code>+</code>), multiplication (<code>*</code>), and parentheses (<code>(...)</code>). Just like normal math, parentheses indicate that the expression inside must be evaluated before it can be used by the surrounding expression. Addition still finds the sum of the numbers on both sides of the operator, and multiplication still finds the product.</p>
<p>However, the rules of <em>operator precedence</em> have changed. Rather than evaluating multiplication before addition, the operators have the <em>same precedence</em>, and are evaluated left-to-right regardless of the order in which they appear.</p>
<p>For example, the steps to evaluate the expression <code>1 + 2 * 3 + 4 * 5 + 6</code> are as follows:</p>
<pre><code><em>1 + 2</em> * 3 + 4 * 5 + 6
  <em>3   * 3</em> + 4 * 5 + 6
      <em>9   + 4</em> * 5 + 6
         <em>13   * 5</em> + 6
             <em>65   + 6</em>
                 <em>71</em>
</code></pre>
<p>Parentheses can override this order; for example, here is what happens if parentheses are added to form <code>1 + (2 * 3) + (4 * (5 + 6))</code>:</p>
<pre><code>1 + <em>(2 * 3)</em> + (4 * (5 + 6))
<em>1 +    6</em>    + (4 * (5 + 6))
     7      + (4 * <em>(5 + 6)</em>)
     7      + <em>(4 *   11   )</em>
     <em>7      +     44</em>
            <em>51</em>
</code></pre>
<p>Here are a few more examples:</p>
<ul>
<li><code>2 * 3 + (4 * 5)</code> becomes <em><code>26</code></em>.</li>
<li><code>5 + (8 * 3 + 9 + 3 * 4 * 3)</code> becomes <em><code>437</code></em>.</li>
<li><code>5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))</code> becomes <em><code>12240</code></em>.</li>
<li><code>((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2</code> becomes <em><code>13632</code></em>.</li>
</ul>
<p>Before you can help with the homework, you need to understand it yourself. <em>Evaluate the expression on each line of the homework; what is the sum of the resulting values?</em></p>
</article>


In [21]:
tests = [
    {
        "name": "Example 1",
        "expression": "1 + 2 * 3 + 4 * 5 + 6",
        "expected": 71,
    },
    {
        "name": "Example 2",
        "expression": "1 + (2 * 3) + (4 * (5 + 6))",
        "expected": 51,
    },
    {
        "name": "Example 3",
        "expression": "2 * 3 + (4 * 5)",
        "expected": 26,
    },
    {
        "name": "Example 4",
        "expression": "5 + (8 * 3 + 9 + 3 * 4 * 3)",
        "expected": 437,
    },
    {
        "name": "Example 5",
        "expression": "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) ",
        "expected": 12240,
    },
    {
        "name": "Example 6",
        "expression": "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2",
        "expected": 13632,
    },
]


def eval(expression: str) -> int:
    value_stack = [0]
    operator_stack = ["+"]

    for ch in expression.strip().replace(" ", ""):
        if ch.isdigit():
            op = operator_stack.pop()
            if op == "+":
                value_stack[-1] += int(ch)
            else:
                value_stack[-1] *= int(ch)
        elif ch == "(":
            value_stack.append(0)
            operator_stack.append("+")
        elif ch == ")":
            value = value_stack.pop()
            op = operator_stack.pop()
            if op == "+":
                value_stack[-1] += value
            else:
                value_stack[-1] *= value
        elif ch == "+":
            operator_stack.append(ch)
        elif ch == "*":
            operator_stack.append(ch)
        else:
            raise ValueError(f"Unknown Character: {ch=}")

    return value_stack[-1]


run_tests_params(eval, tests)


Test Example 1 passed, for eval.
Test Example 2 passed, for eval.
Test Example 3 passed, for eval.
Test Example 4 passed, for eval.
Test Example 5 passed, for eval.
Test Example 6 passed, for eval.
Success


In [22]:
example = """
1 + 2 * 3 + 4 * 5 + 6
1 + (2 * 3) + (4 * (5 + 6))
2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 
"""

expected = 71 + 51 + 26 + 437 + 12240 + 13632


def sum_of_resulting_values(s: str, eval=eval) -> int:
    return sum(eval(e) for e in s.strip().splitlines())


assert sum_of_resulting_values(example) == expected

In [23]:
with open("../input/day18.txt") as f:
    puzzle = f.read()

print(f"Part I: {sum_of_resulting_values(puzzle)}")

Part I: 3348222486398


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>3348222486398</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You manage to answer the child's questions and they finish part 1 of their homework, but get stuck when they reach the next section: <em>advanced</em> math.</p>
<p>Now, addition and multiplication have <em>different</em> precedence levels, but they're not the ones you're familiar with. Instead, addition is evaluated <em>before</em> multiplication.</p>
<p>For example, the steps to evaluate the expression <code>1 + 2 * 3 + 4 * 5 + 6</code> are now as follows:</p>
<pre><code><em>1 + 2</em> * 3 + 4 * 5 + 6
  3   * <em>3 + 4</em> * 5 + 6
  3   *   7   * <em>5 + 6</em>
  <em>3   *   7</em>   *  11
     <em>21       *  11</em>
         <em>231</em>
</code></pre>
<p>Here are the other examples from above:</p>
<ul>
<li><code>1 + (2 * 3) + (4 * (5 + 6))</code> still becomes <em><code>51</code></em>.</li>
<li><code>2 * 3 + (4 * 5)</code> becomes <em><code>46</code></em>.</li>
<li><code>5 + (8 * 3 + 9 + 3 * 4 * 3)</code> becomes <em><code>1445</code></em>.</li>
<li><code>5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))</code> becomes <em><code>669060</code></em>.</li>
<li><code>((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2</code> becomes <em><code>23340</code></em>.</li>
</ul>
<p><em>What do you get if you add up the results of evaluating the homework problems using these new rules?</em></p>
</article>

</main>


In [30]:
from collections import deque
from curses.ascii import isdigit


tests = [
    {
        "name": "Example 0",
        "expression": "1 + 2 + 4 + 6",
        "expected": 13,
    },
    {
        "name": "Example 1",
        "expression": "1 + 2 * 3 + 4 * 5 + 6",
        "expected": 231,
    },
    {
        "name": "Example 2",
        "expression": "1 + (2 * 3) + (4 * (5 + 6))",
        "expected": 51,
    },
    {
        "name": "Example 3",
        "expression": "2 * 3 + (4 * 5)",
        "expected": 46,
    },
    {
        "name": "Example 4",
        "expression": "5 + (8 * 3 + 9 + 3 * 4 * 3)",
        "expected": 1445,
    },
    {
        "name": "Example 5",
        "expression": "5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4)) ",
        "expected": 669060,
    },
    {
        "name": "Example 6",
        "expression": "((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2",
        "expected": 23340,
    },
]


def eval_II(expression: str) -> int:
    values, operators = [], []

    for token in expression.strip().replace(" ", ""):
        if token.isdigit():
            values.append(int(token))
        elif token == "(":
            operators.append(token)
        elif token == ")":
            while operators and operators[-1] != "(":
                operator = operators.pop()
                right = values.pop()
                left = values.pop()
                if operator == "+":
                    value = right + left
                else:
                    value = right * left
                values.append(value)

            operators.pop()

        elif token in "*":
            while operators and operators[-1] not in "()*":
                operator = operators.pop()
                right = values.pop()
                left = values.pop()
                value = right + left
                values.append(value)

            operators.append(token)

        elif token in "+":
            operators.append(token)

    while operators:
        operator = operators.pop()
        right = values.pop()
        left = values.pop()

        if operator == "+":
            value = right + left
        else:
            value = right * left

        values.append(value)

    return values[0]


run_tests_params(eval_II, tests)


Test Example 0 passed, for eval_II.
Test Example 1 passed, for eval_II.
Test Example 2 passed, for eval_II.
Test Example 3 passed, for eval_II.
Test Example 4 passed, for eval_II.
Test Example 5 passed, for eval_II.
Test Example 6 passed, for eval_II.
Success


In [31]:
example = """
1 + 2 * 3 + 4 * 5 + 6
1 + (2 * 3) + (4 * (5 + 6))
2 * 3 + (4 * 5)
5 + (8 * 3 + 9 + 3 * 4 * 3)
5 * 9 * (7 * 3 * 3 + 9 * 3 + (8 + 6 * 4))
((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2 
"""

expected = 231 + 51 + 46 + 1445 + 669060 + 23340


assert sum_of_resulting_values(example, eval_II) == expected

In [32]:
print(f"Part II: {sum_of_resulting_values(puzzle, eval_II)}")

Part II: 43423343619505


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>43423343619505</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>


In [27]:
# 1 + 2 * 3 + 4 * 5 + 6
#   3   * 3 + 4 * 5 + 6
#       9   + 4 * 5 + 6
#          13   * 5 + 6
#              65   + 6
#                  71

# 1 + (2 * 3) + (4 * (5 + 6))
# 1 +    6    + (4 * (5 + 6))
#      7      + (4 * (5 + 6))
#      7      + (4 *   11   )
#      7      +     44
#             51

#  ((2 + 4 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2
#  ((6 * 9) * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2
#  (54 * (6 + 9 * 8 + 6) + 6) + 2 + 4 * 2
#  (54 * (15 * 8 + 6) + 6) + 2 + 4 * 2
#  (54 * (15 * 14) + 6) + 2 + 4 * 2
#  (54 * 210 + 6) + 2 + 4 * 2
#  (54 * 216) + 2 + 4 * 2
#  11664 + 2 + 4 * 2
#  11666 + 4 * 2
#  11670 * 2
#  23340

In [28]:
11670 * 2

23340